In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# All the datasets 

box_office_mojo = pd.read_csv("../data/bom.movie_gross.csv")

imdb_name = pd.read_csv("../data/name.basics.csv")
imdb_title = pd.read_csv("../data/title.basics.csv")
imdb_crew = pd.read_csv("../data/title.crew.csv") 
imdb_principals = pd.read_csv("../data/title.principals.csv")
imdb_ratings = pd.read_csv("../data/title.ratings.csv")
imdb_akas = pd.read_csv("../data/title.akas.csv")

rt_info = pd.read_csv("../data/rt.movie_info.tsv", sep = "\t")
rt_reviews = pd.read_csv("../data/rt.reviews.tsv", sep = "\t", encoding='windows-1252')

the_movie_db = pd.read_csv("../data/tmdb.movies.csv")

movie_budgets = pd.read_csv("../data/tn.movie_budgets.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/bom.movie_gross.csv'

In [ ]:
title_and_ratings = imdb_title.merge(imdb_ratings, on="tconst")

In [ ]:
title_and_ratings = title_and_ratings.rename(columns={"primary_title":"movie"})

In [ ]:
imdb_crew = imdb_crew.rename(columns={"directors":"nconst"})

In [ ]:
names_of_director = imdb_crew.merge(imdb_name, on="nconst")

In [ ]:
imdb_df = title_and_ratings.merge(names_of_director, on="tconst")

In [ ]:
df = imdb_df.merge(movie_budgets, on="movie")

In [ ]:
df = df.drop([#'tconst', 
         'original_title', 
         'start_year', 
         'nconst', 
         'writers', 
         'birth_year', 
         'death_year', 
         'primary_profession', 
         'known_for_titles', 
         'id'], axis=1)

In [ ]:
df.head()

### Clean up columns
1. movie: find duplicates, but check if totally the same, or other features are different
2. runtime_minutes:
3. genres: list multiple genres (in analysis, include combination of genres as separate category)
4. averagerating: rename with underscore
5. numvotes: rename with underscore
6. primary_name:
7. release_data: change format to datetime
8. production_budget: change to int
9. domestic_gross: change to int
10. worldwide_gross: change to int

In [ ]:
df['release_date'] = pd.to_datetime(df['release_date'])

In [ ]:
def dollar_to_int(column):
    return column.str.replace('$', '').str.replace(',', '').map(int)

In [ ]:
df[['production_budget', 'domestic_gross', 'worldwide_gross']] = df[['production_budget', 'domestic_gross', 'worldwide_gross']].apply(dollar_to_int)

In [ ]:
df.head()

### Remove missing values of worldwide_gross

In [ ]:
df = df[~(df['worldwide_gross']==0)]

### Create new columns
11. total_profit
12. profit_budget_ratio
13. day
14. month
15. year

In [ ]:
df['total_profit'] = df['worldwide_gross'] - df['production_budget']

In [ ]:
df['profit_to_budget'] = (df['worldwide_gross'] / df['production_budget'])*100

In [ ]:
df['day'] = df['release_date'].dt.day_name()

In [ ]:
df['month'] = pd.DatetimeIndex(df['release_date']).month

In [ ]:
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### Analysis of release_date: day, month, year

In [ ]:
df['day'].value_counts(normalize=True)*100

In [ ]:
fig, ax = plt.subplots()

day_values = df['day'].value_counts()

ax.barh(
    y=day_values.index,
    width=day_values.values
)
ax.set_ylabel('day');

In [ ]:
df['day'].hist();

In [ ]:
df['month'].value_counts(normalize=True)*100

In [ ]:
fig, ax = plt.subplots()

month_values = df['month'].value_counts()

ax.barh(
    y=month_values.index,
    width=month_values.values
)
ax.set_ylabel('month');

In [ ]:
df['month'].hist();

In [ ]:
df['year'].value_counts(normalize=True)*100

In [ ]:
fig, ax = plt.subplots()

year_values = df['year'].value_counts()

ax.bar(
    x=year_values.index,
    height=year_values.values
)
ax.set_xlabel('year');

In [ ]:
df['month'].hist();

In [ ]:
pd.plotting.scatter_matrix(df[['total_profit', 'profit_to_budget', 'month', 'year']]);

### Relationship between release_date and total_profit

In [ ]:
df.plot('day', 'total_profit', kind='scatter');

In [ ]:
df.plot('month', 'total_profit', kind='scatter');

In [ ]:
df.plot('year', 'total_profit', kind='scatter');

### Relationship between release_date and profit_to_budget

In [ ]:
df.plot('day', 'profit_to_budget', kind='scatter');

In [ ]:
df.plot('month', 'profit_to_budget', kind='scatter');

In [ ]:
df.plot('year', 'profit_to_budget', kind='scatter');

#### Correlation between profit_to_budget and columns

In [ ]:
df.dtypes

In [ ]:
corr_dict = {}
for col in df.columns:
    if col == 'profit_to_budget' or col == 'release_date':
        continue
    if df[col].dtype != 'object':
        col_corr = df['profit_to_budget'].corr(df[col], method='pearson')
        #np.corrcoef(df['profit_to_budget'], df[col])[0][1]
        corr_dict[col] = col_corr
        
max_corr = max(corr_dict, key=corr_dict.get)

In [ ]:
max_corr 

In [ ]:
min_corr = min(corr_dict, key=corr_dict.get)
min_corr

In [ ]:
corr_dict